In [5]:
import numpy as np
data_name = 'NYCBike1' #'NYCBike1'

In [6]:

data_train = np.load(f'./{data_name}/train.npz')
for file in data_train.files:
    print(file, data_train[file].shape)
'''
time.npz
x (3023, 19)
y (3023, 1) [0, 24*7) tod+dow 20000101-00:00 - 20000101-01:00 **** 20000107-23:00
y (3023, 1) [0, 24*7) tod+dow       0 -              1        ****       24*7-1
'''
'''
PEMS0x/METR-LA [0, 24*7*12)
'''

x (3023, 19, 128, 2)
y (3023, 1, 128, 2)
x_offsets (19, 1)
y_offsets (1, 1)


In [7]:

data_val = np.load(f'./{data_name}/val.npz')
for file in data_val.files:
    print(file, data_val[file].shape)

x (431, 19, 128, 2)
y (431, 1, 128, 2)
x_offsets (19, 1)
y_offsets (1, 1)


In [8]:

data_test = np.load(f'./{data_name}/test.npz')
for file in data_test.files:
    print(file, data_test[file].shape)

x (864, 19, 128, 2)
y (864, 1, 128, 2)
x_offsets (19, 1)
y_offsets (1, 1)


In [9]:

data_adj = np.load(f'./{data_name}/adj_mx.npz')
for file in data_adj.files:
    print(file, data_adj[file].shape)
    print(data_adj[file])
# Save to graph.txt
# 对于NYCBike1(16*8), graph的node顺序按照h16w8的gird顺序
np.savetxt('graph.txt', data_adj['adj_mx'], fmt='%d')

adj_mx (128, 128)
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
